In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
import os, sys
import time, timeit
import csv
from sklearn.utils import shuffle


# from joblib import dump, load

# from IPython.display import Image, display
# from six import StringIO
# from sklearn.tree import export_graphviz
max_cardinality = 6400.0

In [2]:
def read_date(file_name):
    df = pd.read_csv(file_name, header=None)
    df = df[df[0] <= max_cardinality]
#     print(df)
    train_x = df[[0,1,2,3,4,5,6,7,8,9]]    
    train_y1 = df[[10]]    
    train_y2 = df[[11]]
    return train_x, train_y1, train_y2

In [3]:
def build_cost_model(train_x, train_y1, train_y2, name):
    build_models = {
        "rfr" : RandomForestRegressor(max_depth=1, n_estimators=20, random_state=0),
        "rfc" : RandomForestClassifier(max_depth=1, random_state=0),
        "dtr" : DecisionTreeRegressor(max_depth=1),
        "dtc" : DecisionTreeClassifier(max_depth=1),
        "nn" : MLPRegressor(hidden_layer_sizes=(32,), activation='relu', learning_rate='constant', learning_rate_init=0.01,  max_iter=500)
    }
    query_models = {
        "rfr" : RandomForestRegressor(max_depth=1, n_estimators=20, random_state=0),
        "rfc" : RandomForestClassifier(max_depth=1, random_state=0),
        "dtr" : DecisionTreeRegressor(max_depth=1),
        "dtc" : DecisionTreeClassifier(max_depth=1),
        "nn" : MLPRegressor(hidden_layer_sizes=(32,), activation='relu', learning_rate='constant', learning_rate_init=0.01,  max_iter=500)
    }
    if name == "rfr" or name == "dtr" or name == "nn":
        train_y1 = train_y1 / 6
        train_y2 = train_y2 / 6
    train_x[0] /= max_cardinality
    return build_models[name].fit(train_x, train_y1), query_models[name].fit(train_x, train_y2)

In [4]:
def predict_score(models, name, xs, lamb):
    build_model, query_model = models[name]
#     if name == "nn":
#         print( build_model.predict(xs) * lamb, query_model.predict(xs) * (1 - lamb))
    return build_model.predict(xs) * lamb + query_model.predict(xs) * (1 - lamb)

In [5]:
file_name = 'train_set_formatted.csv'     
train_x, train_y1, train_y2 = read_date(file_name)
models = {
    "rfr" : build_cost_model(train_x, train_y1, train_y2, "rfr"),
    "rfc" : build_cost_model(train_x, train_y1, train_y2, "rfc"),
    "dtr" : build_cost_model(train_x, train_y1, train_y2, "dtr"),
    "dtc" : build_cost_model(train_x, train_y1, train_y2, "dtr"),
    "nn" : build_cost_model(train_x, train_y1, train_y2, "nn")
}


/tmp/ipykernel_3776381/2986489242.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return build_models[name].fit(train_x, train_y1), query_models[name].fit(train_x, train_y2)
/tmp/ipykernel_3776381/2986489242.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return build_models[name].fit(train_x, train_y1), query_models[name].fit(train_x, train_y2)
/tmp/ipykernel_3776381/2986489242.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return build_models[name].fit(train_x, train_y1), query_models[name].fit(train_x, train_y2)
/tmp/ipykernel_3776381/2986489242.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

In [6]:
class Record:
    def __init__(self, record):
        self.cardinality = int(record[0])
        self.distribution = list(map(int, record[1:4]))
        self.method = list(map(int, record[4:10]))      
        self.build_score = int(record[10])        
        self.query_score = int(record[11])
    
    def is_same(self, record):
        if self.cardinality != record.cardinality:
            return False
        if self.distribution != record.distribution:
            return False
        if self.method != record.method:
            return False
        return True
            

    def __str__(self):
        return  str(self.cardinality) + "-" + str(self.distribution) + "-" + str(self.method)+ "-" +str(self.build_score) + "-" + str(self.build_score) + "\n"
    
    def __repr__(self):
                return  str(self.cardinality) + "-" + str(self.distribution) + "-" + str(self.method)+ "-" +str(self.build_score) + "-" + str(self.build_score) + "\n"

In [7]:
def read_records(input_data_file):
    records = []
    with open(input_data_file) as csvfile:
        reader = csv.reader(csvfile, delimiter = ',')
        for i, row in enumerate(reader):
            record = Record(row)
            records.append(record)
    return records

In [8]:
def find_record(target, records):
    for record in records:
        if record.is_same(target):
            return record
    return None

In [9]:
def test_model(records, models):
    distributions = {'normal':[1,0,0],'skewed':[0,1,0],'uniform':[0,0,1]}
    methods = {'cl':[1,0,0,0,0,0],'mr':[0,1,0,0,0,0],'original':[0,0,1,0,0,0],
                'rl':[0,0,0,1,0,0],'rs':[0,0,0,0,1,0],'sp':[0,0,0,0,0,1]}
    cardinality = [1, 2, 4, 8, 16, 32, 64, 128, 200, 400, 600, 800, 1600, 3200, 6400]
    lambdas = [0.1 * i for i in range(11)]
    model_names = models.keys()
    res = {}
    for name in model_names:
        res[name] = []
    for lamb in lambdas:
        model_accuracy = {}
        total_num = 0
        for name in model_names:
            model_accuracy[name] = 0
        for card in cardinality:
#             if card > max_cardinality:
#                 continue
            for dist in distributions.keys():
                # here find a method
                best_method = ''
                max_score = -100
                for method in methods.keys():
                    record = []
                    record.append(card)
                    record.extend(distributions[dist])
                    record.extend(methods[method])
                    record.append(0)
                    record.append(0)
                    temp = Record(record)
                    result = find_record(temp, records)
                    if result is None:
                        continue
                    current_score = result.build_score * lamb + result.query_score * (1 - lamb)  
                    
                    if current_score > max_score:
                        max_score = current_score
                        best_method = method
                if best_method == '':
                    continue
#                 print("lambda", lamb, "cardinality", card, "distribution", dist, "method", best_method)
                total_num += 1
                for name in model_names:
                    max_score = -100
                    choosed_method = ''
                    for method in methods.keys():
                        record = []
                        record.append(card / max_cardinality)
                        record.extend(distributions[dist])
                        record.extend(methods[method])
                        current_score = predict_score(models, name, [record], lamb)
#                         print("current_score", current_score)
                        
                        if current_score > max_score:
                            max_score = current_score
                            choosed_method = method
#                     print(name, choosed_method, best_method, choosed_method == best_method)
                    if best_method == choosed_method:
                        model_accuracy[name] = model_accuracy[name] + 1
        for name in model_names:
            model_accuracy[name] = round(model_accuracy[name] / total_num, 2)
        print(model_accuracy)
        for name in model_names:
            res[name].append(model_accuracy[name])
    for name in model_names:
        print(res[name])
            
test_model(read_records(file_name), models)  

{'rfr': 0.78, 'rfc': 0.1, 'dtr': 0.78, 'dtc': 0.78, 'nn': 0.78}
{'rfr': 0.78, 'rfc': 0.78, 'dtr': 0.78, 'dtc': 0.78, 'nn': 0.78}
{'rfr': 0.78, 'rfc': 0.78, 'dtr': 0.78, 'dtc': 0.78, 'nn': 0.78}
{'rfr': 0.85, 'rfc': 0.85, 'dtr': 0.85, 'dtc': 0.85, 'nn': 0.85}
{'rfr': 0.83, 'rfc': 0.83, 'dtr': 0.83, 'dtc': 0.83, 'nn': 0.83}
{'rfr': 0.29, 'rfc': 0.66, 'dtr': 0.29, 'dtc': 0.29, 'nn': 0.66}
{'rfr': 0.46, 'rfc': 0.49, 'dtr': 0.46, 'dtc': 0.46, 'nn': 0.49}
{'rfr': 0.76, 'rfc': 0.15, 'dtr': 0.76, 'dtc': 0.76, 'nn': 0.76}
{'rfr': 0.98, 'rfc': 0.98, 'dtr': 0.98, 'dtc': 0.98, 'nn': 0.98}
{'rfr': 1.0, 'rfc': 1.0, 'dtr': 1.0, 'dtc': 1.0, 'nn': 1.0}
{'rfr': 1.0, 'rfc': 1.0, 'dtr': 1.0, 'dtc': 1.0, 'nn': 1.0}
[0.78, 0.78, 0.78, 0.85, 0.83, 0.29, 0.46, 0.76, 0.98, 1.0, 1.0]
[0.1, 0.78, 0.78, 0.85, 0.83, 0.66, 0.49, 0.15, 0.98, 1.0, 1.0]
[0.78, 0.78, 0.78, 0.85, 0.83, 0.29, 0.46, 0.76, 0.98, 1.0, 1.0]
[0.78, 0.78, 0.78, 0.85, 0.83, 0.29, 0.46, 0.76, 0.98, 1.0, 1.0]
[0.78, 0.78, 0.78, 0.85, 0.83, 0.66, 